Uploading readings from Rapsberry Pi to the DataBase

Creating a table for data to go in to

In [4]:
import boto3

dynamodb = boto3.resource('dynamodb')

table = dynamodb.create_table(
    TableName='trial-blackout-monitor-minutes',
    KeySchema=[
        {
            'AttributeName': 'pk',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'sk',
            'KeyType': 'RANGE'
        }
    ],
    AttributeDefinitions=[
        {
            "AttributeName": "pk",
            "AttributeType": "S"
        },
        {
            "AttributeName": "sk",
            "AttributeType": "S"
        }
    ],
    ProvisionedThroughput = {
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    }
)

Code to add a reading from Raspberry Pi to the new table (trial-blackout-monitor-minutes)

In [31]:
import RPi.GPIO as GPIO
import boto3
import datetime
import platform

READ_PIN=4
GPIO.setmode(GPIO.BCM)
GPIO.setup(0, GPIO.OUT)
GPIO.output(0,GPIO.HIGH)
GPIO.setup(READ_PIN,GPIO.IN, pull_up_down = GPIO.PUD_DOWN)
input_io = GPIO.input(4)

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('trial-blackout-monitor-minutes')



table.put_item(
    Item={
        "pk": f"device#{platform.node()}#sensor{READ_PIN}#date#{datetime.datetime.now(datetime.timezone.utc).strftime('%Y-%m-%d-%H')}",
        "sk": f"time#{datetime.datetime.now(datetime.timezone.utc).strftime('%M')}",
        "pk_status": f"device#{platform.node()}#sensor{READ_PIN}#status#done",
        "reading": input_io,
        "ts": f"{datetime.datetime.now(datetime.timezone.utc).isoformat()}"

    }
)



{'ResponseMetadata': {'RequestId': 'ITH8KS8PVLROEHSVTUE6ABRCMFVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Thu, 14 Aug 2025 12:04:46 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ITH8KS8PVLROEHSVTUE6ABRCMFVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}